# Jupyter Notebook containing code for the IBM Capstone Project Course

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Code for the Week 3 Peer-review Assignment

## From here on, the code refers to the segmentation and clustering of the neighborhoods of Toronto

# #### 1st part ####

In [3]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Since the information in the Wikipedia is already in a table (structured),  we can use pandas to directly read this information.

In [4]:
# Reading Wiki page into pandas object
temp_wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
toronto_df = temp_wiki[0]

# Rename the columns to be like the ones shown in the assignment instructions
toronto_df.columns = ['PostalCode','Borough','Neighborhood']

# Drop non-assigned boroughs by first substituting the string by NaN and then droping it
toronto_df.replace('Not assigned', np.NaN, inplace=True)
toronto_df.dropna(inplace=True)
toronto_df.reset_index(inplace=True, drop=True)
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
# Check dataframe's shape
toronto_df.shape

(103, 3)

# #### 2nd part

In [6]:
# Installing and importing geocoder library
!pip install geocoder
import geocoder
print('Library successfully loaded!')

  Using cached https://files.pythonhosted.org/packages/4f/6b/13166c909ad2f2d76b929a4227c952630ebaf0d729f6317eb09cbceccbab/geocoder-1.38.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/29/c1/24814557f1d22c56d50280771a17307e6bf87b70727d975fd6b2ce6b014a/requests-2.25.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ee/ff/48bde5c0f013094d729fe4b0316ba2a24774b3ff1c52d924a8a4cb04078a/six-1.15.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f2/98/7e6d147fd16a10a5f821db6e25f192265d6ecca3d82957a4fdd592cad49c/ratelim-0.1.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/19/c7/fa589626997dd07bd87d9269342ccb74b1720384a4d739a1872bd84fbe68/chardet-4.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5e/a

### WARNING: The geocoder library is not working due to a lack of API key to make requests to Google Maps.
### Since this API key is now paid, I will directly download the .csv file containing latitude and longitude data

In [7]:
# Loop through each postal code to get the latitude and longitude
# As described in the assignment instructions, we should loop through each one because sometimes the request fail

# Create a list containing the lat. and long. for each neighborhood so we can add to the dataframe later
#lat_list=[]
#long_list=[]
#for i, pc in enumerate(toronto_df['PostalCode']):
#    ll_coords=None
#    while (ll_coords == None):
#        print('Trying to get coordinates for {} postal code'.format(pc))
#        g = geocoder.google('{}, Toronto, Ontario'.format(pc))
#        ll_coords = g.latlng
#    lat_list.append(ll_coords[0])
#    long_list.append(ll_coords[1])
    
#lat_list
#long_list

In [8]:
# Download .csv file containing latitude and longitude information (geocoder not working)
!wget -q -O 'latlong_data.csv' https://cocl.us/Geospatial_data
latlong_df = pd.read_csv('latlong_data.csv')
latlong_df.columns = ['PostalCode', latlong_df.columns[1], latlong_df.columns[2]]
latlong_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Join dataframes in order to add lat. and long. columns to the main dataframe

In [9]:
toronto_df = toronto_df.join(latlong_df.set_index('PostalCode'), on='PostalCode')
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [10]:
# Check shape
toronto_df.shape

(103, 5)

# ### 3rd part

### With the latitude and longitude of the postal codes, we can start exploring all these neighborhood using the Foursqure API

### First, we set up the credentials

In [17]:
client_id = 'XSSMD5IEFHMJT0UTBQ2U1SW4IBFS1H04GZG4TXXAL1IEMOVX'
client_secret = '53DFQ1DWAR20AL1QUVAU2CQNROGA13MCNKTLBZ3HNODVVUM3'
version = '20180605'
limit = 100

### Now we loop through each postal code and find the venues within 500 meters of each, for a limit of 100 

### With the retrieved info, we create our working dataframe

In [15]:
import requests
from pandas.io.json import json_normalize

In [18]:
rad = 500
temp_df = toronto_df.set_index('PostalCode')
base_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&radius={}&limit={}'.format(client_id,client_secret,version,rad,limit)

venues_list=[]
for pc in toronto_df['PostalCode']:
    lat = temp_df.loc[pc, 'Latitude']
    long = temp_df.loc[pc, 'Longitude']
    
    # Set up the url for the API request by adding the lat. and long. info to the base url
    url = base_url + '&ll={},{}'.format(lat,long)

    result = requests.get(url).json()['response']['groups'][0]['items']
    
    for venue in result:
        venues_list.append([pc, venue['venue']['name'], venue['venue']['categories'][0]['name']])

In [19]:
venues_df = pd.DataFrame(venues_list)
venues_df.columns=['PostalCode','Venue Name','Venue Category']
venues_df.head()

,PostalCode,Venue Name,Venue Category
0,M3A,Brookbanks Park,Park
1,M3A,Brookbanks Pool,Pool
2,M3A,Variety Store,Food & Drink Shop
3,M4A,Victoria Village Arena,Hockey Arena
4,M4A,Portugril,Portuguese Restaurant


### Now we create dummie variables/features (one-hot encoding) for each category so we can calculate the frequency (and mean) of each venue for each postal code
### Remember that we are doing all in terms of postal codes because they are the ones holding the latitude/longitude information, not the boroughs
### We may experiment grouping the dataframe by boroughs later for the clustering

In [20]:
# First, let's count the number of unique categories
len(pd.unique(venues_df['Venue Category']))

270

In [21]:
# One-hot enconding
venues_onehot = pd.get_dummies(venues_df[['Venue Category']], prefix="", prefix_sep="")

# We now add the postal codes
venues_onehot['PostalCode'] = venues_df['PostalCode']

# Fix columns
col_order = [[venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])]
venues_onehot = venues_onehot[col_order[0]]
venues_onehot.head(10)

,PostalCode,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [22]:
# Take the mean of category appearances for each postal code
venues_grouped = venues_onehot.groupby('PostalCode').mean().reset_index()
venues_grouped.head()

,PostalCode,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

### With our working dataframe created, we are now ready to apply the K-Means algorithms

In [23]:
from sklearn.cluster import KMeans

In [24]:
venues_cluster = venues_grouped.drop('PostalCode', 1)

kclusters = 5

kmm = KMeans(n_clusters=kclusters, init="k-means++", n_init=10, random_state=0)

kmm.fit(venues_cluster)

kmm.labels_

array([2, 0, 2, 0, 2, 1, 0, 2, 2, 2, 2, 2, 0, 2, 4, 0, 2, 0, 2, 4, 0, 4,
       0, 4, 2, 0, 0, 0, 4, 2, 2, 0, 0, 2, 2, 2, 0, 2, 4, 0, 2, 0, 2, 2,
       0, 0, 1, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0,
       0, 0, 0, 0, 2, 2, 4, 2, 2, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 0,
       2, 2, 2, 3, 0, 2, 2, 2, 0, 2, 2, 0], dtype=int32)

### With the labels obtained, we insert it into the previous dataframe

In [25]:
venues_grouped.insert(1, 'Cluster', kmm.labels_)
venues_grouped.head()

,PostalCode,Cluster,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

### Let's clean the database, dropping the categories and adding latitude/longitude so we can finally plot it

In [26]:
# Drop the category information
venues_grouped = venues_grouped.iloc[:,[0,1]]

# Add latitude/longitude and borough/neighborhood information
postal_clusters = venues_grouped.join(toronto_df.set_index('PostalCode'), on='PostalCode')

# Rearrange columns so that cluster goes to the end
new_cols = [col for col in postal_clusters.columns if not col == 'Cluster']
new_cols.append('Cluster')

postal_clusters = postal_clusters[new_cols]
postal_clusters.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2


### Now we can finally create a Folium map and insert our postal code-based clusters

In [27]:
import folium

import matplotlib.pyplot as plt
import matplotlib.colors as colors_mpl

In [28]:
toronto_lat = 43.6532
toronto_long = -79.3832

# Create our map centered in Toronto
postal_map = folium.Map(location=[toronto_lat, toronto_long], zoom_start=11)

# Now we create our colored markers for each postal code and cluster number
cols = plt.cm.Spectral(np.linspace(0,1,len(set(kmm.labels_))))
cols = [colors_mpl.rgb2hex(x) for x in cols]

for postal, lat, long, clus, bor in zip(postal_clusters['PostalCode'], postal_clusters['Latitude'], postal_clusters['Longitude'], postal_clusters['Cluster'], postal_clusters['Borough']):
    label = folium.Popup('{}: {}, {}'.format(postal,bor,clus))
    folium.CircleMarker([lat,long],
                       radius=5,
                       popup=label,
                       fill=True,
                       color=cols[clus],
                       fill_color=cols[clus],
                       fill_opacity=0.9).add_to(postal_map)
postal_map

### We can see that most of Toronto's postal codes are clustered as "0"
### We could then analyze what are the characteristics of the 0-cluster so that we can make assertions about these neighborhoods

## From the above result, we see that clustering by postal codes may not be the best choice, since many different postal codes have very similar lat/long. locations

## We now try something different
### We will segment and cluster only the neighborhoods pertaining to Downtown Toronto borough

In [29]:
# We first form our dataframe containing Downtown Toronto information
dt_df = toronto_df[toronto_df['Borough']=='Downtown Toronto']
dt_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


### Now we explore venues for each (set of) neighborhood in a radius of 200 meters

In [30]:
rad = 200
temp_df = dt_df.set_index('Neighborhood')
base_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&radius={}&limit={}'.format(client_id,client_secret,version,rad,limit)

dtvenues_list=[]
for nb in dt_df['Neighborhood']:
    lat = temp_df.loc[nb, 'Latitude']
    long = temp_df.loc[nb, 'Longitude']
    
    # Set up the url for the API request by adding the lat. and long. info to the base url
    url = base_url + '&ll={},{}'.format(lat,long)
    
    result = requests.get(url).json()['response']['groups'][0]['items']
    
    for venue in result:
        dtvenues_list.append([nb, venue['venue']['name'], venue['venue']['categories'][0]['name']])

In [31]:
# We now create a dataframe with the venues found from the above list
dtvenues_df = pd.DataFrame(dtvenues_list, columns=['Neighborhood', 'Venue Name', 'Venue Category'])
dtvenues_df.head()

,Neighborhood,Venue Name,Venue Category
0,"Regent Park, Harbourfront",Tandem Coffee,Coffee Shop
1,"Regent Park, Harbourfront",Roselle Desserts,Bakery
2,"Regent Park, Harbourfront",Body Blitz Spa East,Spa
3,"Regent Park, Harbourfront",Morning Glory Cafe,Breakfast Spot
4,"Regent Park, Harbourfront",The Extension Room,Gym / Fitness Center


In [32]:
# One-hot encoding of the above df
dtvenues_onehot = pd.get_dummies(dtvenues_df[['Venue Category']], prefix="", prefix_sep="")

# Now we add the Neighborhood and fix columns
dtvenues_onehot['Neighborhood'] = dtvenues_df['Neighborhood']
col_order = ['Neighborhood'] + [col for col in dtvenues_onehot.columns if not col == 'Neighborhood']
dtvenues_onehot = dtvenues_onehot[col_order]
dtvenues_onehot.head()

,Neighborhood,Adult Boutique,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Belgian Restaurant,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Convenience Store,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Dumpling Restaurant,Escape Room,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,History Museum,Hotel,Hotel Bar,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Lake,Liquor Store,Market,Martial Arts School,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,Movie Theater,Music Venue,New American Restaurant,Noodle House,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoothie Shop,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theme Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Finally, we get the mean frequency for each neighborhood and apply our K-Means algorithm

In [33]:
dtvenues_mean = dtvenues_onehot.groupby('Neighborhood').mean().reset_index()
dtvenues_mean.head()

,Neighborhood,Adult Boutique,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Belgian Restaurant,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Convenience Store,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Dumpling Restaurant,Escape Room,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,History Museum,Hotel,Hotel Bar,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Lake,Liquor Store,Market,Martial Arts School,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,Movie Theater,Music Venue,New American Restaurant,Noodle House,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoothie Shop,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theme Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
1,Central Bay Street,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.071429,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.071429,0.0,0.0,0.000000,0.000000,0.0,0.428571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.071429,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.071429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.071429,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,Christie,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3,Church and Wellesley,0.047619,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.047619,0.047619,0.047619,0.0,0.095238,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.047619,0.047619,0.0,0.047619,0.0,0.0,0.000000,0.047619,0.0,0

In [34]:
dtvenues_cluster = dtvenues_mean.drop('Neighborhood', axis=1)

kclusters=5

dt_kmm = KMeans(init="k-means++", n_clusters=kclusters, n_init=10, random_state=0)
dt_kmm.fit(dtvenues_cluster)

dt_kmm.labels_

array([3, 1, 4, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1], dtype=int32)

### Now we once again clean the dataframe and insert appropriate information for plotting

In [35]:
dtvenues_mean.insert(1, 'Cluster', dt_kmm.labels_)
dtvenues_mean.head()

,Neighborhood,Cluster,Adult Boutique,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Belgian Restaurant,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Convenience Store,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Dumpling Restaurant,Escape Room,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,History Museum,Hotel,Hotel Bar,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Lake,Liquor Store,Market,Martial Arts School,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,Movie Theater,Music Venue,New American Restaurant,Noodle House,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoothie Shop,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theme Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,"CN Tower, King and Spadina, Railway Lands, Har...",3,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
1,Central Bay Street,1,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.071429,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.071429,0.0,0.0,0.000000,0.000000,0.0,0.428571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.071429,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.071429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.071429,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,Christie,4,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3,Church and Wellesley,1,0.047619,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.047619,0.047619,0.047619,0.0,0.095238,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.047619,0.047619,0.0,0.047619,0.0,0.0,0.00000

In [36]:
dtcluster_df = dtvenues_mean.iloc[:, :2]
dtcluster_df.head()

,Neighborhood,Cluster
0,"CN Tower, King and Spadina, Railway Lands, Har...",3
1,Central Bay Street,1
2,Christie,4
3,Church and Wellesley,1
4,"Commerce Court, Victoria Hotel",1


In [37]:
# Join Postal Code and Lat/Long information
dtcluster_df = dtcluster_df.join(dt_df.drop('Borough',axis=1).set_index('Neighborhood'), on='Neighborhood')

# Let Cluster column be the last one and PostalCode the first, just for the sake of consistency
col_order = ['PostalCode'] + [col for col in dtcluster_df.columns if not (col=='PostalCode' or col=='Cluster')] + ['Cluster']
dtcluster_df = dtcluster_df[col_order]
dtcluster_df.head()

,PostalCode,Neighborhood,Latitude,Longitude,Cluster
0,M5V,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,3
1,M5G,Central Bay Street,43.657952,-79.387383,1
2,M6G,Christie,43.669542,-79.422564,4
3,M4Y,Church and Wellesley,43.665860,-79.383160,1
4,M5L,"Commerce Court, Victoria Hotel",43.648198,-79.379817,1


### Finally, we plot the neighborhoods with cluster colors on a Folium map

In [38]:
# Let us centralize the map in Downtown Toronto area
# In order to do that, we get the mean of lat/long from all postal codes in such area
lat,long = float(dt_df.groupby('Borough').mean()['Latitude']), float(dt_df.groupby('Borough').mean()['Longitude'])

dt_map = folium.Map(location=[lat,long], zoom_start=13)

# Now we create our colored markers for each postal code and cluster number
cols = plt.cm.Spectral(np.linspace(0,1,len(set(dt_kmm.labels_))))
cols = [colors_mpl.rgb2hex(x) for x in cols]

for postal, lat, long, clus, nb in zip(dtcluster_df['PostalCode'], dtcluster_df['Latitude'], dtcluster_df['Longitude'], dtcluster_df['Cluster'], dtcluster_df['Neighborhood']):
    label = folium.Popup('{}: {}, {}'.format(postal,nb,clus))
    folium.CircleMarker([lat,long],
                       radius=5,
                       popup=label,
                       fill=True,
                       color=cols[clus],
                       fill_color=cols[clus],
                       fill_opacity=0.9).add_to(dt_map)
dt_map

###### We can see that, since this area is very dense, and hence the Neighborhoods are too close to each other, almost all of them pertain to a single cluster